Copyright **`(c)`** 2024 Giovanni Squillero `<giovanni.squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free under certain conditions — see the [`license`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  

In [701]:
import logging
from itertools import combinations,permutations
from dataclasses import dataclass
import pandas as pd
import numpy as np
import geopy.distance
import random
import copy

from tqdm import tqdm
from icecream import ic

logging.basicConfig(level=logging.DEBUG)

## Lab2 - TSP



In [702]:
cities = pd.read_csv('cities/italy.csv', header=None, names=['name', 'lat', 'lon'])
cities.head()

N_STEPS = 100_000

In [703]:
dist_matrix = np.zeros((len(cities), len(cities)))
for c1, c2 in combinations(cities.itertuples(), 2):
    dist_matrix[c1.Index, c2.Index] = dist_matrix[c2.Index, c1.Index] = geopy.distance.geodesic(
        (c1.lat, c1.lon), (c2.lat, c2.lon)
    ).km

In [704]:
@dataclass
class Individual:
    genome:list[int]
    fitness:float=None

def fitness(individual:Individual)->int:
    tot_cost = 0
    for c1, c2 in zip(individual.genome, individual.genome[1:]):
        tot_cost += dist_matrix[c1, c2]
    return tot_cost


## Crossover Algorithms

In [705]:
def crossover(parent1: Individual, parent2: Individual) -> Individual:
    cut_point = np.random.randint(len(parent1.genome))

    offspring = [None] * len(parent1.genome)
    
    genome_p1 = parent1.genome[:cut_point]
    offspring[:cut_point] = genome_p1
    
    genome_p2 = parent2.genome
    fill_idx = cut_point
    
    for gene in genome_p2:
        if gene not in offspring:
            offspring[fill_idx] = gene
            fill_idx += 1
    
    offspring = [gene for gene in offspring if gene is not None]
    offspring.append(offspring[0])

    
    return Individual(offspring)

def PMX_crossover(parent1:Individual, parent2:Individual)->tuple[Individual,Individual]:
    cut_1, cut_2 = sorted(np.random.choice(len(cities), size=2, replace=False))

    offspring1 = [None] * len(cities)
    offspring2 = [None] * len(cities)

    genome_p1 = parent1.genome.copy()[:-1]
    genome_p2 = parent2.genome.copy()[:-1]

    offspring1[cut_1:cut_2] = genome_p2[cut_1:cut_2]
    offspring2[cut_1:cut_2] = genome_p1[cut_1:cut_2]

    boundary_idx = list(range(0,cut_1)) + list(range(cut_2,len(cities)))
    
    for i in boundary_idx:
        if genome_p1[i] not in offspring1:
            offspring1[i] = genome_p1[i]
        else:
            tmp = genome_p1[i]
            j = i
            while tmp in offspring1:
                j = offspring1.index(genome_p1[j])
                tmp = offspring2[j]
            offspring1[i] = tmp

    for i in boundary_idx:
        if genome_p2[i] not in offspring2:
            offspring2[i] = genome_p2[i]
        else:
            tmp = genome_p2[i]
            j = i
            while tmp in offspring2:
                j = offspring2.index(genome_p2[j])
                tmp = offspring1[j]
            offspring2[i] = tmp
    
    offspring1 = [gene for gene in offspring1 if gene is not None]
    offspring2 = [gene for gene in offspring2 if gene is not None]
    offspring1.append(offspring1[0])
    offspring2.append(offspring2[0])
    return Individual(offspring1), Individual(offspring2)

## Mutation Algorithms

In [706]:
def mutation(p:Individual,multi_mutation=False)->Individual:
    genome = p.genome.copy()
    x = 0
    while x < 0.5:
        idx_city1 = np.random.randint(len(cities))
        idx_city2 = np.random.randint(len(cities))

        genome[idx_city1],genome[idx_city2] = genome[idx_city2],genome[idx_city1]

        x = np.random.random()
        if not multi_mutation:
            break
    return Individual(genome)

def mutationRSMPopulation(population:list[Individual])->list[Individual]: #reverse sequence mutation over the entire population
    offspring=copy.deepcopy(population)
    for i in range(len(population)):
        idx1 = np.random.randint(1,len(population[i].genome)-1)
        idx2 = np.random.randint(1,len(population[i].genome)-1)
        while idx1 == idx2:
            idx2 = np.random.randint(1,len(population[i].genome)-1)
        low,high=sorted([idx1,idx2])
        offspring[i].genome[low:high] = offspring[i].genome[high-1:low-1:-1]
    return offspring

def mutate_whole_population(population:list[Individual], mutation_rate:float)->list[Individual]:
    mutated_population = copy.deepcopy(population)
    # mutated_population = population[:]
    for i in range(len(mutated_population)):
        has_mutated = False
        mutation_probabilities = [np.random.random()< mutation_rate for _ in range(len(cities))]
        mutation_points = [i for i in range(len(cities)) if mutation_probabilities[i]]
        if len(mutation_probabilities)>0:
            for point in mutation_points:
                idx_city1 = point
                idx_city2 = np.random.randint(len(cities))

                mutated_population[i].genome[idx_city1],mutated_population[i].genome[idx_city2] = mutated_population[i].genome[idx_city2],mutated_population[i].genome[idx_city1]              
            
                has_mutated = True

        if has_mutated:
            mutated_population[i].fitness = fitness(mutated_population[i])
    return mutated_population

In [707]:
a = [1,2,3,4,5,6]
low = 1
high = 5
print(a[low:high])
print(a[high-1:low-1:-1])

[2, 3, 4, 5]
[5, 4, 3, 2]


In [708]:
def parent_selection(population:list[Individual]):
    #tournament challengers
    challengers = np.random.choice(population,2)
    candidates = sorted(challengers, key=lambda e: e.fitness)
    return candidates[0]

def parent_selection_rank(population: list[Individual], num_parents: int) -> list[Individual]:
    #assigning ranks based on fitness  
    sorted_population = sorted(population, key=lambda ind: ind.fitness)
    population_size = len(population)
    ranks = np.arange(1, population_size + 1)
    
    # selecting parents based on the rank probabilities
    total_rank_sum = sum(ranks)
    probabilities = [(population_size - rank + 1) / total_rank_sum for rank in ranks]  #linear rank-based probability
    selected_parents_indices = np.random.choice(np.arange(population_size), size=num_parents, replace=False, p=probabilities)
    
    selected_parents = [sorted_population[idx] for idx in selected_parents_indices]
    
    return selected_parents

In [709]:
a = [1,2,3,4,5,6]
print(a[-1])

6


In [710]:
def get_starting_solution()->Individual:
    """
    Compute a greedy sub-optimal solution
    """
    visited = np.full(len(cities), False)
    dist = dist_matrix.copy()
    city = 0
    visited[city] = True
    tsp = list()
    tsp.append(int(city))
    a=0

    while not np.all(visited):
        dist[:, city] = np.inf
        closest = np.argmin(dist[city])
        # logging.debug(
        #     f"step: {cities.at[city,'name']} -> {cities.at[closest,'name']} ({dist_matrix[city,closest]:.2f}km)"
        # )
        visited[closest] = True
        city = closest
        tsp.append(int(city))
    # logging.debug(
    #     f"step: {cities.at[tsp[-1],'name']} -> {cities.at[tsp[0],'name']} ({dist_matrix[tsp[-1],tsp[0]]:.2f}km)"
    # )
    tsp.append(tsp[0])
    return Individual(tsp)

def generate_solution_from_existing(parent:Individual)->Individual:
    new_genome = random.sample(parent.genome[:-1],len(parent.genome[:-1]))
    new_genome.append(new_genome[0])
    return Individual(new_genome)

In [711]:
e = get_starting_solution()
e2 = generate_solution_from_existing(e)
offspring1= crossover(e, e2)
# print(e.genome)
# print(e2.genome)
assert len(offspring1.genome[:-1]) == len(set(offspring1.genome))


### Evolutionary Algorithm

In [712]:
# We start from an already sub-optimal greedy solution and then we try to optimize it further
starting_individual = get_starting_solution()
tot_cost = fitness(starting_individual)
starting_individual.fitness=tot_cost
ic(tot_cost)
best_sol_so_far = starting_individual

POPULATION_SIZE = 100
OFFSPRING_SIZE= POPULATION_SIZE//2
VALHALLA_RATE = 0.2

population = [generate_solution_from_existing(starting_individual) for _ in range(POPULATION_SIZE-1)]
population.append(starting_individual)


for elem in population:
    elem.fitness = fitness(elem)
take_over=False
offspring = list()
for generation in tqdm(range(10_000)):
    offspring = []

    offspring = mutationRSMPopulation(random.sample(population,OFFSPRING_SIZE))

    for i in offspring:
        i.fitness = fitness(i)

    if np.random.random() < VALHALLA_RATE and best_sol_so_far not in population:
        population.append(best_sol_so_far)

    population.extend(offspring)
    population.sort(key=lambda i: i.fitness)
    population = population[:POPULATION_SIZE]

    if not take_over:
        take_over = all([population[0].genome == e.genome for e in population])
        if take_over:
            ic("Takeover at iteration",generation)
    if population[0].fitness < best_sol_so_far.fitness:
        best_sol_so_far = population[0]
            
    # for e in population:
    #     print(e.genome)
    # print("-")
# for e in population:
#     print(e)
# ic(best_sol_so_far.fitness)
print(best_sol_so_far.fitness)

ic| tot_cost: np.float64(4436.03176952516)
100%|██████████| 10000/10000 [00:26<00:00, 372.77it/s]

4181.6197997589
